# Word Sense Disambiguation


### Exercises 1
Even though NLTK states that it implements Original Lesk Algorithm, in fact it is a Simplified Lesk Algorithm, that doesn't consider examples, and computes overlaps like the original. 

In the original algorithm context is computed differently. <mark style="background-color: rgba(0, 255, 0, 0.2)">Instead of comparing a target word's signature with the context words, the target signature is compared with the signatures of each of the context words. </mark>

Implement the Original Lesk Algorithm (modifying NLTK's, see pseudocode above)
Todo list:
- Complete lesk simplified
- Preprocessing:
    - compute pos-tag with `nltk.pos_tag`
    - remove stopwords
        - `from nltk.corpus import stopwords`
        - `stopwords.words('english')`

- take the majority decision (the sense predicted most frequently)

POS tags reminder:

| POS | in Synset Name |
|:----|:---------------|
| `wn.NOUN`    | `n`
| `wn.VERB`    | `v`
| `wn.ADV`     | `r`
| `wn.ADJ`     | `a`
| `wn.ADJ_SAT` | `s` (satelite adjective, ignore)


In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def preprocess(text):
    mapping = {"NOUN": wordnet.NOUN, "VERB": wordnet.VERB, "ADJ": wordnet.ADJ, "ADV": wordnet.ADV}
    sw_list = stopwords.words('english')
    
    lem = WordNetLemmatizer()
    
    # tokenize, if input is text
    tokens = nltk.word_tokenize(text) if type(text) is str else text
    # pos-tag
    tagged = nltk.pos_tag(tokens, tagset="universal")
    # lowercase
    tagged = [(w.lower(), p) for w, p in tagged]
    # optional: remove all words that are not NOUN, VERB, ADJ, or ADV (i.e. no sense in WordNet)
    tagged = [(w, p) for w, p in tagged if p in mapping]
    # re-map tags to WordNet (return orignal if not in-mapping, if above is not used)
    tagged = [(w, mapping.get(p, p)) for w, p in tagged]
    # remove stopwords
    tagged = [(w, p) for w, p in tagged if w not in sw_list]
    # lemmatize
    tagged = [(w, lem.lemmatize(w, pos=p), p) for w, p in tagged]
    # unique the list
    tagged = list(set(tagged))
    return tagged

In [ ]:
def get_sense_definitions(context):
    # input is text or list of strings
    lemma_tags = preprocess(context)
    # let's get senses for each
    senses = [(w, wordnet.synsets(l, p)) for w, l, p in lemma_tags]
    
    # let's get their definitions
    definitions = []
    for raw_word, sense_list in senses:
        if len(sense_list) > 0:
            # let's tokenize, lowercase & remove stop words 
            def_list = []
            for s in sense_list:
                defn = s.definition()
                # let's use the same preprocessing
                tags = preprocess(defn)
                toks = [l for w, l, p in tags]
                def_list.append((s, toks))
            definitions.append((raw_word, def_list))
    return definitions
    

In [ ]:
def get_top_sense(words, sense_list):
    # get top sense from the list of sense-definition tuples
    # assumes that words and definitions are preprocessed identically
    val, sense = max((len(set(words).intersection(set(defn))), ss) for ss, defn in sense_list)
    return val, sense

In [ ]:
from collections import Counter
def lesk_simplified(context_sentence, ambiguous_word, pos=None, synsets=None):
    context = set(context_sentence)
    
    if synsets is None:
        synsets = wordnet.synsets(ambiguous_word)
    if pos:
        synsets = [ss for ss in synsets if str(ss.pos()) == pos]

    if not synsets:
        return None
    # Measure the overlap between context and definitions
    _, sense = max(
        (len(context.intersection(ss.definition().split())), ss) for ss in synsets
    )

    return sense

def original_lesk(context_sentence, ambiguous_word, pos=None, synsets=None, majority=False):

    context_senses = get_sense_definitions(set(context_sentence)-set([ambiguous_word]))
    if synsets is None:
        synsets = get_sense_definitions(ambiguous_word)[0][1]

    if pos:
        synsets = [ss for ss in synsets if str(ss[0].pos()) == pos]

    if not synsets:
        return None
    scores = []
    # print(synsets)
    for senses in context_senses:
        for sense in senses[1]:
            scores.append(get_top_sense(sense[1], synsets))
            
    if len(scores) == 0:
        return synsets[0][0]
    
    if majority:
        filtered_scores = [x[1] for x in scores if x[0] != 0]
        if len(filtered_scores) > 0:
            best_sense = Counter(filtered_scores).most_common(1)[0][0]
        else:
            # Almost random selection
            best_sense = Counter([x[1] for x in scores]).most_common(1)[0][0]
    else:
        _, best_sense = max(scores)
    return best_sense



In [ ]:
text = "Jane sat on the sloping bank of a river beside the water".split()
word = "bank"
print("Sense from lesk original", original_lesk(text, word, majority=True))
print("Sense from lesk simplified", lesk_simplified(text, word))
print("Sense from lesk NLTK", lesk(text, word))


### Exercise 2
Extend Lesk algorithm (function) to use similarity metrics instead of just overlaps
- add a keyword argument to allow different metrics

Complete the Predersen algorithm with the similiratiy metrics
- compare the output of the two algorithms

In [ ]:
semcor_ic = wordnet_ic.ic('ic-semcor.dat')
def get_top_sense_sim(context_sense, sense_list, similarity):
    # get top sense from the list of sense-definition tuples
    # assumes that words and definitions are preprocessed identically
    scores = []
    for sense in sense_list:
        ss = sense[0]
        if similarity == "path":
            try:
                scores.append((context_sense.path_similarity(ss), ss))
            except:
                scores.append((0, ss))    
        elif similarity == "lch":
            try:
                scores.append((context_sense.lch_similarity(ss), ss))
            except:
                scores.append((0, ss))
        elif similarity == "wup":
            try:
                scores.append((context_sense.wup_similarity(ss), ss))
            except:
                scores.append((0, ss))
        elif similarity == "resnik":
            try:
                scores.append((context_sense.res_similarity(ss, semcor_ic), ss))
            except:
                scores.append((0, ss))
        elif similarity == "lin":
            try:
                scores.append((context_sense.lin_similarity(ss, semcor_ic), ss))
            except:
                scores.append((0, ss))
        elif similarity == "jiang":
            try:
                scores.append((context_sense.jcn_similarity(ss, semcor_ic), ss))
            except:
                scores.append((0, ss))
        else:
            print("Similarity metric not found")
            return None
    val, sense = max(scores)
    return val, sense

def lesk_similarity(context_sentence, ambiguous_word, similarity="resnik", pos=None, 
                    synsets=None, majority=True):
    context_senses = get_sense_definitions(set(context_sentence) - set([ambiguous_word]))
    
    if synsets is None:
        synsets = get_sense_definitions(ambiguous_word)[0][1]

    if pos:
        synsets = [ss for ss in synsets if str(ss[0].pos()) == pos]

    if not synsets:
        return None
    
    scores = []
    
    # Here you may have some room for improvement
    # For instance instead of using all the definitions from the context
    # you pick the most common one of each word (i.e. the first)
    for senses in context_senses:
        for sense in senses[1]:
            scores.append(get_top_sense_sim(sense[0], synsets, similarity))
            
    if len(scores) == 0:
        return synsets[0][0]
    
    if majority:
        filtered_scores = [x[1] for x in scores if x[0] != 0]
        if len(filtered_scores) > 0:
            best_sense = Counter(filtered_scores).most_common(1)[0][0]
        else:
            # Almost random selection
            best_sense = Counter([x[1] for x in scores]).most_common(1)[0][0]
    else:
        _, best_sense = max(scores)
    
    return best_sense

def pedersen(context_sentence, ambiguous_word, similarity="resnik", pos=None, 
                    synsets=None, threshold=0.1):
    
    context_senses = get_sense_definitions(set(context_sentence) - set([ambiguous_word]))

    if synsets is None:
        synsets = get_sense_definitions(ambiguous_word)[0][1]

    if pos:
        synsets = [ss for ss in synsets if str(ss[0].pos()) == pos]

    if not synsets:
        return None
    
    synsets_scores = {}
    for ss_tup in synsets:
        ss = ss_tup[0]
        if ss not in synsets_scores:
            synsets_scores[ss] = 0
        for senses in context_senses:
            scores = []
            for sense in senses[1]:
                if similarity == "path":
                    try:
                        scores.append((sense[0].path_similarity(ss), ss))
                    except:
                        scores.append((0, ss))    
                elif similarity == "lch":
                    try:
                        scores.append((sense[0].lch_similarity(ss), ss))
                    except:
                        scores.append((0, ss))
                elif similarity == "wup":
                    try:
                        scores.append((sense[0].wup_similarity(ss), ss))
                    except:
                        scores.append((0, ss))
                elif similarity == "resnik":
                    try:
                        scores.append((sense[0].res_similarity(ss, semcor_ic), ss))
                    except:
                        scores.append((0, ss))
                elif similarity == "lin":
                    try:
                        scores.append((sense[0].lin_similarity(ss, semcor_ic), ss))
                    except:
                        scores.append((0, ss))
                elif similarity == "jiang":
                    try:
                        scores.append((sense[0].jcn_similarity(ss, semcor_ic), ss))
                    except:
                        scores.append((0, ss))
                else:
                    print("Similarity metric not found")
                    return None
            value, sense = max(scores)
            if value > threshold:
                synsets_scores[sense] = synsets_scores[sense] + value
    
    values = list(synsets_scores.values())
    if sum(values) == 0:
        print('Warning all the scores are 0')
    senses = list(synsets_scores.keys())
    best_sense_id = values.index(max(values))
    return senses[best_sense_id]


In [ ]:
text = "Jane sat on the sloping bank of a river beside the water".split()
word = "bank"
sense = original_lesk(text, word, majority=True)
print('Original lesk', sense, sense.definition())
sense = lesk(text, word)
print('Symplified lesk', sense, sense.definition())
sense = predersen(text, word, similarity="resnik", threshold=0.1)
print("Pedersen", sense, sense.definition())
sense = lesk_similarity(text, word, pos='n', similarity="path")
print('Graph-based lesk', sense, sense.definition())


### Exercise 3
- Evaluate Original Lesk (your implementation on Senseval's `interest`)
- You can also easily evaluate Lesk similarity that we have seen before

In [ ]:
from nltk.metrics.scores import precision, recall, f_measure, accuracy

refs = {k: set() for k in mapping.values()}
hyps = {k: set() for k in mapping.values()}
refs_list = []
hyps_list = []

# since WordNet defines more senses, let's restrict predictions

synsets = []
for ss in wordnet.synsets('interest', pos='n'):
    if ss.name() in mapping.values():
        defn = ss.definition()
        tags = preprocess(defn)
        toks = [l for w, l, p in tags]
        synsets.append((ss,toks))

for i, inst in enumerate(senseval.instances('interest.pos')):
    txt = [t[0] for t in inst.context]
    raw_ref = inst.senses[0] # let's get first sense
    hyp = original_lesk(txt, txt[inst.position], synsets=synsets, majority=True).name()
    ref = mapping.get(raw_ref)
    
    # for precision, recall, f-measure        
    refs[ref].add(i)
    hyps[hyp].add(i)
    
    # for accuracy
    refs_list.append(ref)
    hyps_list.append(hyp)

print("Acc:", round(accuracy(refs_list, hyps_list), 3))

for cls in hyps.keys():
    p = precision(refs[cls], hyps[cls])
    r = recall(refs[cls], hyps[cls])
    f = f_measure(refs[cls], hyps[cls], alpha=1)
    
    print("{:15s}: p={:.3f}; r={:.3f}; f={:.3f}; s={}".format(cls, p, r, f, len(refs[cls])))